In [68]:
import pandas as pd
import pickle
from pymongo import MongoClient
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np

In [51]:
collection = 'covid_county_formatted'

db = MongoClient("lattice-100", 27018)
no_of_records = 1064950

pipeline = [
    {
        "$sample": {"size": no_of_records // 10}
    }
]

cursor = db.sustaindb[collection].aggregate(pipeline)
df_original = pd.DataFrame(list(cursor))

In [52]:
df = df_original[["GISJOIN", "cases"]]
df

,GISJOIN,cases
0,G2001650,0
1,G0801170,17
2,G1600870,5
3,G5500630,29
4,G3600790,67
...,...,...
106490,G1700130,0
106491,G5300730,5
106492,G0600410,10
106493,G2001030,0


## Aggregate on 'cases' for each GISJoin

In [77]:
df1 = pd.DataFrame(df.groupby("GISJOIN")["cases"].sum())
dfX = df1['cases']
df1

,cases
GISJOIN,
G0100010,546
G0100030,2751
G0100050,192
G0100070,223
G0100090,437
...,...
G5600370,397
G5600390,273
G5600410,124


In [78]:
dfX = np.array(dfX).reshape(-1, 1)

In [79]:
kmeans = KMeans(n_clusters=56, random_state=0).fit(dfX)
df_predict = kmeans.fit_predict(dfX)
centroids = kmeans.cluster_centers_
df1['cluster_id'] = df_predict
df1

,cases,cluster_id
GISJOIN,,
G0100010,546,23
G0100030,2751,37
G0100050,192,43
G0100070,223,43
G0100090,437,53
...,...,...
G5600370,397,53
G5600390,273,34
G5600410,124,9


In [109]:
len(centroids)

56

In [106]:
cluster_ids_list = df1['cluster_id']
centroids_column = []

for id in cluster_ids_list:
    centroids_column.append(centroids[id][0])
    
print(len(centroids_column))



3115

In [107]:
df1['centroid'] = centroids_column

In [114]:
df1['distance'] = abs(df1['cases'] - df1['centroid'])
df1

,cases,cluster_id,centroid,distance
GISJOIN,,,,
G0100010,546,23,505.491935,40.508065
G0100030,2751,37,2857.650000,106.650000
G0100050,192,43,190.920000,1.080000
G0100070,223,43,190.920000,32.080000
G0100090,437,53,411.383648,25.616352
...,...,...,...,...
G5600370,397,53,411.383648,14.383648
G5600390,273,34,260.253275,12.746725
G5600410,124,9,126.590206,2.590206


In [121]:
df_final = df1[["cluster_id", "distance"]]
df_final

,cluster_id,distance
GISJOIN,,
G0100010,23,40.508065
G0100030,37,106.650000
G0100050,43,1.080000
G0100070,43,32.080000
G0100090,53,25.616352
...,...,...
G5600370,53,14.383648
G5600390,34,12.746725
G5600410,9,2.590206


In [124]:
df_final.to_csv('~/ucc-21/clusters-covid.csv')

Index(['G0100010', 'G0100030', 'G0100050', 'G0100070', 'G0100090', 'G0100110',
       'G0100130', 'G0100150', 'G0100170', 'G0100190',
       ...
       'G5600270', 'G5600290', 'G5600310', 'G5600330', 'G5600350', 'G5600370',
       'G5600390', 'G5600410', 'G5600430', 'G5600450'],
      dtype='object', name='gis_join', length=3115)